In [17]:
import pandas as pd
import numpy as np

data_raw = pd.read_csv('../raw/지역사회건강조사 1차 전처리_2019.csv')

In [18]:
data_pbhlth_code = pd.read_excel('../raw/지역사회건강조사 2019년 원시자료 이용지침서 등/지역사회건강조사 2019 원시자료 참고사항_보건소정보, 세대유형.xlsx',header=3)
data_pbhlth_code = data_pbhlth_code[data_pbhlth_code['시도코드'] == 11][['보건소코드','보건소명']]
data_pbhlth_code["보건소명"] = data_pbhlth_code["보건소명"].str.replace('보건소', '', regex=True)
data_pbhlth_code.columns=['보건소코드','자치구']
list_region = list(data_pbhlth_code.to_dict()['자치구'].values()) + ['서울시']
list_region.reverse()

In [19]:
df = data_raw[['만나이','성별','자치구', '걷기 실천 일수', '걷기 실천 시간(시)','걷기 실천 시간(분)']]

In [20]:
df1=df[df['걷기 실천 일수'] != 99]
df2=df1[df1['걷기 실천 일수'] != 77]

In [21]:
def transtime(df_walk) :
    if 0 <= df_walk['걷기 실천 시간(시)'] <= 24 and df_walk['걷기 실천 시간(분)'] <= 59 :
        return df_walk['걷기 실천 시간(시)']*60 + df_walk['걷기 실천 시간(분)']
    elif df_walk['걷기 실천 시간(시)'] > 24 and df_walk['걷기 실천 시간(분)'] <= 59 :
        return df_walk['걷기 실천 시간(분)']
    else :
        return 0
    
def health_index(df_walk) :
    if df_walk['걷기 실천 일수'] < 5 :
        return 0
    elif 5 <= df_walk['걷기 실천 일수'] and df_walk['걷기 시분 환산'] >= 30 :
        return 1
    else :
        return 0

In [22]:
df_walk = df2.copy() # 오류제거
df_walk["걷기 시분 환산"] = df_walk.apply(transtime, axis =1)
df_walk['걷기 실천'] = df_walk.apply(health_index, axis =1)
df_walk = df_walk.drop(['걷기 실천 시간(시)','걷기 실천 시간(분)','걷기 실천 일수','걷기 시분 환산'],axis=1)

In [23]:
list_df = []
for value in list_region:
    temp = df_activity['자치구'] == value
    list_df.append(df_activity[temp])
list_df[0] = df_activity

In [24]:
list_age = ['19~200','19~44','45~64','65~74','75~200']
list_sex = ['계','남','여']
idx_sex = []
idx_age = []
idx_region = []
value_walk = []
for i, value1 in enumerate(list_region):
    for j in list_age:
        for k in list_sex:            
            temp = list_df[i][list_df[i]['만나이']>=int(j.split('~')[0])]
            temp1 = temp[temp['만나이'] < int(j.split('~')[1])]
            if k == '남':
                temp2 = temp1[temp1['성별'] == 1]
            elif k == '여':
                temp2 = temp1[temp1['성별'] == 2]
            else:
                temp2 = temp1
            temp3 = temp2['걷기 실천'].value_counts(normalize = True).to_dict()
            if 1 not in temp3:
                temp3[1] = 0
            if 0 not in temp3:
                temp3[0] = 0
            value_walk.append(round(temp3[1]*100,1))
            
            if j == '19~200':
                idx_age.append('계')
            elif j == '75~200':
                idx_age.append('75 이상')
            else:
                idx_age.append(j)
            idx_sex.append(k)
            idx_region.append(value1)

In [25]:
df_final = pd.DataFrame({'걷기 실천':value_walk, '지역':idx_region,'연령별':idx_age,'성별':idx_sex})

df_final = df_final.set_index(['지역','연령별','성별'])

df_final

걷기 실천
지역  연령별   성별       
서울시 계     계    59.6
          남    62.3
          여    57.5
    19~44 계    62.9
          남    66.1
...             ...
강남구 65~74 남    56.2
          여    56.4
    75 이상 계    40.0
          남    45.5
          여    36.8

[390 rows x 1 columns]

In [26]:
df_final.to_csv("../raw/지역사회건강조사 (전처리된 데이터)_2019/지역사회건강조사 (걷기)_2019.csv", mode='w',encoding = 'utf-8-sig')